In [8]:
import json
from pathlib import Path
import pysolr

import pandas as pd

PATH = "\\final.pkl"
PATH = str(Path().absolute())+ PATH
fo = open(PATH,"r")

res = pd.read_pickle(PATH)
res = res.loc[res['is_submission'] == False]


In [12]:
slim_df = res[["selftext","title","topic","body","parent_body"]]

def get_response(row):
    if not pd.isna(row["body"]):
        return row["body"]
    if not pd.isna(row["selftext"]):
        return row["selftext"]
    return None

slim_df['response'] = slim_df.apply(lambda row:get_response(row),axis=1)

C:\Users\ashis\AppData\Local\Temp\ipykernel_22436\1323430379.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slim_df['response'] = slim_df.apply(lambda row:get_response(row),axis=1)


In [13]:
slim_df = slim_df[slim_df['parent_body'].notnull()]
slim_df = slim_df[['response','parent_body','topic']]
slim_df['query'] = slim_df['parent_body']
slim_df=slim_df[['query','response','topic']]


In [18]:
import contractions
import re

In [20]:
def cleaner(text):    
    text = contractions.fix(text)
    text = text.replace("&gt;", "")
    text = text.replace("&lt;", "")
    text = text.replace("\n", " ")

    regex_url = r"(http[^\s]+)"
    wo_url = re.sub(regex_url,"",text)
    wo_url = re.sub(r"#","",wo_url)
    # wo_url = re.sub(r"(@[^\s]+)","",wo_url)
    wo_url = re.sub(r"(@[^\s]+)","@USER_",wo_url)

    return wo_url



slim_df['query'] = slim_df['query'].apply(cleaner)
slim_df['response'] = slim_df['response'].apply(cleaner)

In [22]:
slim_df.to_pickle('slim.pkl')
slim_records = slim_df.to_dict('records')

In [16]:
solr = pysolr.Solr('http://34.130.165.83:8983/solr/reddit', always_commit=True,timeout=1100)


# print('cleaned_objs',len(slim_records))
# cleaned_objs = pd.DataFrame(slim_records)
# cleaned_objs.to_pickle('chitchat.pkl')
response = solr.add(slim_records)
# print(response)

fo.close()
